In [1]:
# Autoreload
%load_ext autoreload
%autoreload 2
from glob import glob
import json
import sys
import os
import numpy as np
np.printoptions(decimals=1)

In [2]:
base='../data'
# Find all the json files recursively
json_files = glob(base + '/**/*.json', recursive=True)

print(f'Found {len(json_files)} json files')


Found 1626 json files


In [3]:
json_files = glob(base + '/**/*.json', recursive=True)

defectos_set=set()
for json_file in json_files:
    try:
        with open(json_file, 'r') as f:
            data = json.load(f)
            anotaciones_vistas=data["views_annotations"]
            for k,v in anotaciones_vistas.items():
                defectos=list(v.keys())
                defectos_set.update(defectos)
            
    except:
        print(f'Error reading {json_file}')
print(f'Found {len(defectos_set)} defectos')
print(defectos_set)

Error reading ../data/mandarins/clementine/manfredi2/light_scars/20250402_1323_002040.json
Error reading ../data/mandarins/clementine/manfredi2/light_scars/20250402_1321_000004.json
Error reading ../data/oranges/navel/manfredi2/skin_creasing/20250403_0955_000051.json
Error reading ../data/oranges/navel/manfredi2/skin_creasing/20250403_0954_000041.json
Error reading ../data/oranges/navel/manfredi2/skin_creasing/20250403_0955_000052.json
Error reading ../data/oranges/navel/manfredi2/skin_creasing/20250403_0954_000047.json
Error reading ../data/oranges/navel/manfredi2/skin_creasing/20250403_0955_000049.json
Error reading ../data/oranges/navel/manfredi2/deformed_counterpeduncle/20250403_0938_000030.json
Error reading ../data/oranges/navel/manfredi2/deformed_counterpeduncle/20250403_0937_000027.json
Error reading ../data/oranges/navel/manfredi2/mechanical_damage/20250402_1423_002089.json
Error reading ../data/oranges/navel/manfredi2/mechanical_damage/20250402_1422_002082.json
Error reading 

In [4]:
np.set_printoptions(precision=1)
json_files = glob(base + '/**/*.json', recursive=True)

defectos_set=list(defectos_set)
defectos_set.sort()
cuenta={ defecto:0 for defecto in defectos_set}

cuenta['sin_defecto']=0
cuenta['mas_de_una']=0

cuenta_vistas=0
anotaciones_dataset=[]
lista_sin_views=[]
listas_sospechosos={}
for defecto in defectos_set:
    listas_sospechosos[defecto]=[]
for json_file in json_files:

    with open(json_file, 'r') as f:
        data = json.load(f)
        if "views_annotations" not in data:
            print(json_file,"no views_annotations")
            lista_sin_views.append(json_file)
            continue
            
        anotaciones_vistas=data["views_annotations"]
        #print(anotaciones_vistas)
        anots_array=[]
        for nombre_vista,anotaciones_vista in anotaciones_vistas.items():
            #print(v)
            cuenta_vistas+=1
            
            nada=True
            c=0
            
            anots=[]
            m_defectos=[]
            for defecto,valor in anotaciones_vista.items():
                #print(valor)
                vint=int(valor)
                m_defectos.append(defecto)
                anots.append(vint)
                if vint>0:
                    cuenta[defecto]+=1
                    nada=False
                    c+=1
            if nada:
                cuenta['sin_defecto']+=1
            if c>1:
                cuenta['mas_de_una']+=1
            anots_array.append(anots)
        anots_array=np.array(anots_array)
        
        vistas_con_cada_defecto=np.mean(anots_array,axis=0)
        #print(vistas_con_cada_defecto)
        if np.any(vistas_con_cada_defecto>0.99):
            print(json_file)
            print(m_defectos)
            print(vistas_con_cada_defecto)
            
            def_idx=np.nonzero(vistas_con_cada_defecto>0.99)
            
            
            for idx in def_idx[0]:
                defecto=m_defectos[idx]
                listas_sospechosos[defecto].append(json_file)
                print(f'Adding {json_file} to {defecto}')
            print("=========================")    
        anotaciones_dataset.append(vistas_con_cada_defecto)

            
    # except:
    #     print(f'Error reading {json_file}')
print('Frutos sin vistas',len(lista_sin_views))

for k,v in listas_sospechosos.items():
    v.sort()
    
print("Total frutos: ", len(json_files))
print("Total Vistas: ", cuenta_vistas)
for d in cuenta:
    print(f'Vistas con {d}: {cuenta[d]}')


../data/mandarins/clementine/KDC/20220808_154231_Img_20220808_154156produccionClementina/20250218_1154_002404.json
['bleaching', 'cracks', 'damage_by_pliers', 'deformed_counterpeduncle', 'deformed_peduncle', 'discarded', 'green', 'hollow', 'insect', 'light_scars', 'mechanical_damage', 'oil_spots', 'plu_stickers', 'reddish', 'rotten', 'scars', 'skin_breakdown', 'skin_creasing', 'surface']
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Adding ../data/mandarins/clementine/KDC/20220808_154231_Img_20220808_154156produccionClementina/20250218_1154_002404.json to deformed_peduncle
../data/mandarins/clementine/KDC/20220808_154231_Img_20220808_154156produccionClementina/20250218_1153_002402.json
['bleaching', 'cracks', 'damage_by_pliers', 'deformed_counterpeduncle', 'deformed_peduncle', 'discarded', 'green', 'hollow', 'insect', 'light_scars', 'mechanical_damage', 'oil_spots', 'plu_stickers', 'reddish', 'rotten', 'scars', 'skin_breakdown', 'skin_creasing', 'surface']
[0. 0. 0. 0. 1. 

/home/aalbiol/miniconda3/envs/mscandvc/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3904: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/aalbiol/miniconda3/envs/mscandvc/lib/python3.12/site-packages/numpy/_core/_methods.py:147: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


../data/oranges/navel/manfredi2/light_scars/20250402_1345_000031.json no views_annotations
../data/oranges/navel/manfredi2/light_scars/20250402_1349_002055.json no views_annotations
../data/oranges/navel/manfredi2/light_scars/20250402_1347_002046.json no views_annotations
../data/oranges/navel/manfredi2/light_scars/20250402_1348_002054.json no views_annotations
../data/oranges/navel/manfredi2/deformed_peduncle/20250403_1039_002038.json
['insect', 'cracks', 'light_scars', 'rotten', 'scars', 'green', 'hollow', 'skin_breakdown', 'damage_by_pliers', 'deformed_counterpeduncle', 'oil_spots', 'discarded', 'surface', 'bleaching', 'mechanical_damage', 'skin_creasing', 'deformed_peduncle', 'plu_stickers', 'reddish']
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
Adding ../data/oranges/navel/manfredi2/deformed_peduncle/20250403_1039_002038.json to reddish
../data/oranges/navel/manfredi2/deformed_peduncle/20250403_1040_000047.json
['insect', 'cracks', 'light_scars', 'rotten', 'scars', 

In [5]:
lista_sin_views

['../data/mandarins/clementine/manfredi2/light_scars/20250402_1323_002040.json',
 '../data/mandarins/clementine/manfredi2/light_scars/20250402_1321_000004.json',
 '../data/oranges/navel/manfredi2/skin_creasing/20250403_0955_000051.json',
 '../data/oranges/navel/manfredi2/skin_creasing/20250403_0954_000041.json',
 '../data/oranges/navel/manfredi2/skin_creasing/20250403_0955_000052.json',
 '../data/oranges/navel/manfredi2/skin_creasing/20250403_0954_000047.json',
 '../data/oranges/navel/manfredi2/skin_creasing/20250403_0955_000049.json',
 '../data/oranges/navel/manfredi2/deformed_counterpeduncle/20250403_0938_000030.json',
 '../data/oranges/navel/manfredi2/deformed_counterpeduncle/20250403_0937_000027.json',
 '../data/oranges/navel/manfredi2/mechanical_damage/20250402_1423_002089.json',
 '../data/oranges/navel/manfredi2/mechanical_damage/20250402_1422_002082.json',
 '../data/oranges/navel/manfredi2/light_scars/20250402_1345_000031.json',
 '../data/oranges/navel/manfredi2/light_scars/2025

In [6]:
print("Total frutos: ", len(json_files))
print("Total Vistas: ", cuenta_vistas)
for d in cuenta:
    print(f'Vistas con {d}: {cuenta[d]}')

Total frutos:  1626
Total Vistas:  13948
Vistas con bleaching: 198
Vistas con cracks: 192
Vistas con damage_by_pliers: 183
Vistas con deformed_counterpeduncle: 21
Vistas con deformed_peduncle: 692
Vistas con discarded: 0
Vistas con green: 676
Vistas con hollow: 327
Vistas con insect: 118
Vistas con light_scars: 722
Vistas con mechanical_damage: 210
Vistas con oil_spots: 263
Vistas con plu_stickers: 436
Vistas con reddish: 158
Vistas con rotten: 574
Vistas con scars: 693
Vistas con skin_breakdown: 483
Vistas con skin_creasing: 88
Vistas con surface: 1635
Vistas con sin_defecto: 7414
Vistas con mas_de_una: 1043
